# Visión por Computadora II - TP Integrador

**Dataset**: [fruits-360](https://www.kaggle.com/datasets/moltean/fruits)

In [ ]:
# importamos librerias
import os
import gdown
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix